In [2]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

from datasets import load_dataset
import matplotlib.pyplot as plt
import base64
import io
from PIL import Image

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

openai_client = OpenAI(
    os.getenv('OPENAI_API_KEY')  
)

In [21]:
def get_gpt(messages, model='gpt-4o-mini', response_format="json_object", temperature=0.1):
    try:
        response = openai_client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature,
            response_format={'type' : response_format},
        )
    except Exception as e:
        print(e)
        return False
    
    answer = response.choices[0].message.content.strip()
    if response_format == 'json_object':
        return json.loads(answer)
    return answer

In [6]:
import pandas as pd

splits = {'train': 'train.json', 'test': 'test.json'}
df = pd.read_json("hf://datasets/TableQAKit/WTQ/" + splits["train"])

In [7]:
df_test = pd.read_json("hf://datasets/TableQAKit/WTQ/" + splits["test"])

In [8]:
df_test.head()

,id,question,table_id,table,answer_text,struct_in,text_in,seq_out
0,nu-0,which country had the most cyclists finish wit...,csv/203-csv/733.csv,"{'header': ['Rank', 'Cyclist', 'Team', 'Time',...",[Italy],col : rank | cyclist | team | time | uci proto...,which country had the most cyclists finish wit...,italy
1,nu-1,how many people were murdered in 1940/41?,csv/204-csv/149.csv,"{'header': ['Description Losses', '1939/40', '...","[100,000]",col : description losses | 1939/40 | 1940/41 |...,how many people were murdered in 1940/41?,"100,000"
2,nu-2,how long did it take for the new york american...,csv/203-csv/435.csv,"{'header': ['Year', 'Division', 'League', 'Reg...",[17 years],col : year | division | league | reg. season |...,how long did it take for the new york american...,17 years
3,nu-3,alfie's birthday party aired on january 19. wh...,csv/204-csv/803.csv,"{'header': ['Series #', 'Season #', 'Title', '...","[January 26, 1995]",col : series # | season # | title | notes | or...,alfie's birthday party aired on january 19. wh...,"january 26, 1995"
4,nu-4,what is the number of 1st place finishes acros...,csv/204-csv/272.csv,"{'header': ['Date', 'Competition', 'Location',...",[17],col : date | competition | location | country ...,what is the number of 1st place finishes acros...,17


In [9]:
def dict2Image(data):
    df = pd.DataFrame(data['rows'], columns=data['header'])

    # 테이블 시각화
    fig, ax = plt.subplots(figsize=(10, 4))  # 테이블 크기 조정 (가로 길이를 넉넉히)

    ax.axis('tight')
    ax.axis('off')

    # 테이블 생성 (글씨 크기 키우기)
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    # 테이블 스타일 설정
    table.auto_set_font_size(False)
    table.set_fontsize(10)  # 글씨 크기 설정

    for i in range(len(data['header'])):
        table.auto_set_column_width([i])
    
    # 이미지를 메모리에 저장
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png', bbox_inches='tight', dpi=300)
    plt.close(fig)  # 메모리에서 이미지 제거

    # 메모리에서 base64로 변환
    img_buffer.seek(0)
    img_base64 = base64.b64encode(img_buffer.read()).decode('utf-8')
    img_buffer.close()

    return img_base64

In [22]:
def eval_with_text(data):
    prompt = f"""
Answer the following question. Just give the answer, not the process.

Table: {data['table']}

Question: {data['question']}

Answer: 
"""
    answer = get_gpt(messages=[{'role':'user', 'content': prompt}], response_format = 'text')
    answer = answer.lower()
    print('question: ', data['question'])
    print('gpt: ', answer)
    print('answer: ', data['seq_out'])
    return data['seq_out'] in answer

In [23]:
def eval_with_image(data):
    image = dict2Image(data['table'])
    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": "Answer the following question. Just give the answer, not the process. \nTable: "},
            {
                "type": "image_url",
                "image_url": {
                    "url" : f"data:image/jpeg;base64,{image}",
                    "detail" : 'high'
                }
            },
            {"type": "text", "text": f"Question: {data['question']}\nAnswer:"}
        ],
    }]
    answer = get_gpt(messages, response_format = 'text')
    answer = answer.lower()
    print('question: ', data['question'])
    print('gpt: ', answer)
    print('answer: ', data['seq_out'])
    return data['seq_out'] in answer

In [32]:
data = df_test.iloc[20]
print('------TEXT------')
correct1 = eval_with_text(data)
print(correct1)
print('------IMAGE------')
correct2 = eval_with_image(data)
print(correct2)


base64_string = dict2Image(data.table)
# Base64 문자열을 디코딩하여 바이너리 데이터로 변환
image_data = base64.b64decode(base64_string)

# 메모리에서 이미지 열기
image = Image.open(io.BytesIO(image_data))

# 이미지 보기 (디코딩된 이미지)
image.show()

------TEXT------
question:  what was the number of times won on grass?
gpt:  3
answer:  1
False
------IMAGE------
question:  what was the number of times won on grass?
gpt:  3
answer:  1
False
